In [2]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-dn1i6t02
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-dn1i6t02
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=2151646a8d423863f364b509f900541095ff7f471b07bd2c86ab36007a28000f
  Stored in directory: /tmp/pip-ephem-wheel-cache-2hugnkqb/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import pandas as pd
import os
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optims
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import subprocess

device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()


dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/train_plus_small_2.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/train_plus_small_2.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset('/content/tst.txt', field_indices=[1,2], num_discard_samples=1)


tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)


# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)



class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 231, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)



# model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)
model = torch.load('/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/best_kobert_model_small.pt')

# if torch.cuda.device_count() > 1:
    # model = nn.DataParallel(model)
# model = nn.DataParallel(model, output_device=[0,1])
# model.to(device)



# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]




# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# lr_scheduler = optims.lr_scheduler.CosineAnnealingLR(optimizer,T_max=0.1,eta_min=0.0001)


# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc



# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    best_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        # lr_scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        if train_acc > best_acc:
            best_acc = train_acc
        torch.save(model, '/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/best_kobert_small_2_0522.pt')
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))



torch.save(model, '/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/last_kobert_small_2_0522.pt')


# # model.eval() # 평가 모드로 변경
    
# # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
# #     token_ids = token_ids.long().to(device)
# #     segment_ids = segment_ids.long().to(device)
# #     valid_length= valid_length
# #     label = label.long().to(device)
# #     out = model(token_ids, valid_length, segment_ids)
# #     test_acc += calc_accuracy(out, label)
# # print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

using cached model
using cached model
using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch 1 batch id 1 loss 2.6538610458374023 train acc 0.5
epoch 1 batch id 201 loss 1.2830785512924194 train acc 0.6668221393034826
epoch 1 batch id 401 loss 0.8736941814422607 train acc 0.6671602244389028
epoch 1 batch id 601 loss 0.8923899531364441 train acc 0.6673772878535774
epoch 1 batch id 801 loss 1.5696357488632202 train acc 0.6629993757802747
epoch 1 batch id 1001 loss 1.1472407579421997 train acc 0.6624313186813187
epoch 1 batch id 1201 loss 1.0258311033248901 train acc 0.6626509159034139
epoch 1 batch id 1401 loss 1.970084309577942 train acc 0.6599527123483226
epoch 1 batch id 1601 loss 1.7736555337905884 train acc 0.659236414740787
epoch 1 batch id 1801 loss 1.0070611238479614 train acc 0.6586271515824542
epoch 1 batch id 2001 loss 2.0092527866363525 train acc 0.6582646176911544
epoch 1 batch id 2201 loss 1.2832001447677612 train acc 0.657045093139482
epoch 1 batch id 2401 loss 1.1184011697769165 train acc 0.6558595376926281
epoch 1 batch id 2601 loss 1.6162811517715454 trai

train END

In [ ]:
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/train_plus_small_1.txt', field_indices=[1,2], num_discard_samples=1)

In [ ]:
# dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/train_plus_small_2.txt', field_indices=[1,2], num_discard_samples=1)

In [ ]:
dataset_val = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/val_small.txt', field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 50
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 231, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# model = torch.load('/content/drive/MyDrive/애쓰는 감자/data/minz_data/best_kobert_model_small.pt')

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    val_acc = 0.0
    best_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        if train_acc > best_acc:
           best_acc = train_acc
        torch.save(model, '/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/best_kobert_small_1_0522.pt')
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, val_acc / (batch_id+1)))

    torch.save(model, '/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/last_kobert_small_1_0522.pt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.5504603385925293 train acc 0.5416666666666666
epoch 1 batch id 201 loss 0.6707344651222229 train acc 0.6685323383084572
epoch 1 batch id 401 loss 1.4045881032943726 train acc 0.6626142975893597
epoch 1 batch id 601 loss 1.2170381546020508 train acc 0.664309484193012
epoch 1 batch id 801 loss 0.7562181353569031 train acc 0.6639617145235119
epoch 1 batch id 1001 loss 1.332964301109314 train acc 0.6617965367965366
epoch 1 batch id 1201 loss 1.1418415307998657 train acc 0.6627116291978901
epoch 1 batch id 1401 loss 1.2520266771316528 train acc 0.662324530097549
epoch 1 batch id 1601 loss 1.0380767583847046 train acc 0.6628149073495728
epoch 1 batch id 1801 loss 1.07827889919281 train acc 0.6619239311493613
epoch 1 batch id 2001 loss 1.4770020246505737 train acc 0.6616275195735465
epoch 1 batch id 2201 loss 0.9899908900260925 train acc 0.6612335302135391
epoch 1 batch id 2401 loss 1.138363242149353 train acc 0.6617381646536162
epoch 1 batch id 2601 loss 1.324914097

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.6645824532432132


epoch 2 batch id 1 loss 2.445823907852173 train acc 0.5833333333333334
epoch 2 batch id 201 loss 0.6696441769599915 train acc 0.6648009950248758
epoch 2 batch id 401 loss 1.3822304010391235 train acc 0.661783042394015
epoch 2 batch id 601 loss 1.2818067073822021 train acc 0.663893510815309
epoch 2 batch id 801 loss 0.7930278778076172 train acc 0.6643778610070753
epoch 2 batch id 1001 loss 1.3492579460144043 train acc 0.6617965367965376
epoch 2 batch id 1201 loss 1.1970771551132202 train acc 0.6630585623091874
epoch 2 batch id 1401 loss 1.3659133911132812 train acc 0.6629490839876286
epoch 2 batch id 1601 loss 0.9872567653656006 train acc 0.6628669581511561
epoch 2 batch id 1801 loss 1.0093976259231567 train acc 0.6617851193781247
epoch 2 batch id 2001 loss 1.4782794713974 train acc 0.6609820089955036
epoch 2 batch id 2201 loss 1.025614857673645 train acc 0.6602491291837054
epoch 2 batch id 2401 loss 1.1294909715652466 train acc 0.6609572400388739
epoch 2 batch id 2601 loss 1.3435357809


epoch 2 test acc 0.6645824532432132


<font color = pink> test 예측하기 </font>

In [ ]:
# test 예측하기
device = torch.device('cuda:0')
model = torch.load('/content/drive/MyDrive/애쓰는 감자/data/minz_data/best_kobert_model_small.pt')
model.to(device)
# model = nn.DataParallel(model, output_device=[0,1])
bertmodel, vocab = get_pytorch_kobert_model()
model.eval() # 평가 모드로 변경
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/new_data/bert_test.txt', field_indices=[1,2], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
test_acc = 0.0
df = pd.DataFrame(columns=['pred','label'])
pred = np.array([])
answer = np.array([])
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _,max_idx = torch.max(out,1)
    pred = np.append(pred,max_idx.cpu().detach().tolist())
    # answer = np.append(answer,label.cpu().detach().tolist())
    # test_acc += calc_accuracy(out, label)
    # print(len(pred))
df['pred'] = pred
# df['label'] = answer

df.to_csv('/content/drive/MyDrive/애쓰는 감자/data/minz_data/test_small.csv',index=False)
# print("test acc {}".format(test_acc / (batch_id+1)))

using cached model
using cached model
using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


<font color = pink> val 예측하기 </font>

In [ ]:
# validation 예측하기

device = torch.device('cuda:0')
model = torch.load('/content/drive/MyDrive/애쓰는 감자/data/minz_data/weight/best_kobert_model_small.pt')
model.to(device)
# model = nn.DataParallel(model, output_device=[0,1])
bertmodel, vocab = get_pytorch_kobert_model()
model.eval() # 평가 모드로 변경
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
max_len = 256 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/애쓰는 감자/data/minz_data/val_small_pred.txt', field_indices=[1,2], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
test_acc = 0.0
df = pd.DataFrame(columns=['pred','label'])
pred = np.array([])
answer = np.array([])
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _,max_idx = torch.max(out,1)
    pred = np.append(pred,max_idx.cpu().detach().tolist())
    # answer = np.append(answer,label.cpu().detach().tolist())
    # test_acc += calc_accuracy(out, label)
    # print(len(pred))
df['pred'] = pred
# df['label'] = answer

df.to_csv('/content/drive/MyDrive/애쓰는 감자/data/minz_data/val_small_pred.csv',index=False)
# print("test acc {}".format(test_acc / (batch_id+1)))

using cached model
using cached model
using cached model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
